In [ ]:
#Different EEG Augmentation functions
#!git clone https://github.com/braindecode/braindecode.git

#TorchEEG manuell import:
#!git clone https://github.com/torcheeg/torcheeg.git

#SimCLR TorchEEG:
#https://torcheeg.readthedocs.io/en/latest/generated/torcheeg.trainers.SimCLRTrainer.html#torcheeg.trainers.SimCLRTrainer

In [ ]:
!pip install -r requirements.txt

In [1]:
#Data_paths
DATA_PATH = './' #where data is saved
TMP_DATA = './Tmp_Datasets'
CHECKPOINT_PATH = './saved_models' # Path to the folder where the pretrained models are saved

# Path to the folder where the training dataset is located
SEED_PATH = './SEED/SEED_EEG/ExtractedFeatures/Subj'
SEEDiv_PATH = './SEED/SEED_IV/ExtractedFeatures'

In [2]:
import os
from copy import deepcopy
NUM_WORKERS = os.cpu_count()
# Create checkpoint path if it doesn't exist yet
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

gpus = [0]
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, gpus))
print(f'NUM_WORKERS: {NUM_WORKERS}')

import numpy as np
import io
import itertools
import random
import math
from typing import Dict, List, Union

#Import Pytorch
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.nn.functional import pad
    from torch import Tensor
    from torch.fft import fft, ifft
    import torch.optim as optim
    import sklearn
    import seaborn 

except ModuleNotFoundError: 
    !pip install seaborn 
    !pip install torch
    !pip install scikit-learn
    import seaborn 
    import sklearn
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.nn.functional import pad
    from torch import Tensor
    import torch.optim as optim
    from torch.fft import fft, ifft
# PyTorch Lightning
try:
    import lightning.pytorch as pl
    #import pytorch_lightning as pl
    import torchmetrics
    import torcheeg
    import wandb
    

except ModuleNotFoundError: 
    !pip install wandb
    !pip install comet-ml
    !pip install lightning
    #!pip install --quiet pytorch-lightning>=1.4
    !pip install torcheeg
    !pip install torchmetrics
    import wandb
    #import comet_ml
    import lightning.pytorch as pl
    #import pytorch_lightning as pl
    import torcheeg
    import torchmetrics


from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.loggers import CometLogger
from lightning.pytorch.callbacks import GradientAccumulationScheduler, LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

#from pytorch_lightning.loggers import WandbLogger
#from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint


from typing import Any, Tuple, List
from torcheeg.datasets import SEEDDataset, DEAPDataset
from torcheeg import transforms, model_selection


#Create custom dataset:(inherits from torch.utils.data.Dataset)
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.optimizer import Optimizer, required

from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
import scipy
import scipy.io as scio #for loading raw EEG

import matplotlib
import matplotlib.pyplot as plt
import mne

import PIL

from matplotlib import colors
from mne.viz import circular_layout
from mne_connectivity.viz import plot_connectivity_circle
from pylab import cm
import pathlib
from tqdm import tqdm



#Local imports 
#from Plotting import PlotEEG
#from Datasets import create_dataset
from AllModels import *
#from Augmentations import ContrastiveTransformations

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

NUM_WORKERS: 96


2023-12-05 13:45:05,220 DEBUG MainThread git.cmd Popen(['git', 'version'], cwd=/workspace, stdin=None, shell=False, universal_newlines=False)
2023-12-05 13:45:05,224 DEBUG MainThread git.cmd Popen(['git', 'version'], cwd=/workspace, stdin=None, shell=False, universal_newlines=False)
2023-12-05 13:45:05,238 DEBUG MainThread wandb.docker.auth Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
2023-12-05 13:45:05,240 DEBUG MainThread wandb.docker.auth No config file found
2023-12-05 13:45:05,297 DEBUG MainThread sentry_sdk.errors [Tracing] Create new propagation context: {'trace_id': 'b84c24e16ad1450fa1db9b5daf86aef0', 'span_id': 'ab621656937cc10d', 'parent_span_id': None, 'dynamic_sampling_context': None}


Tue Dec  5 13:45:05 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     On  | 00000000:52:00.0 Off |                    0 |
|  0%   31C    P8              21W / 300W |      4MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
'''
Parameter
'''

#EEG - Data:
chunk_size=200 #length of a sample
channels = 62
overlap=0 

#NonSSL Models:
nr_devices = 'auto'
batch_size=64
nr_classes=3 #Number of target classes
nr_epochs=30
lr=4e-4  # Conformer paper uses 0.0002
weight_decay=1e-3

#SSL-only Model
batch_sizeSSL = 1024 #based on https://arxiv.org/pdf/2002.05709.pdf
lrSSL= 5e-4 #0.0001 
weight_decaySSL = 1e-6 #based on https://arxiv.org/pdf/2002.05709.pdf
temperatureSSL = 0.05 #based on mohsenvand2020a
nr_epochs_ssl=100 

#https://arxiv.org/pdf/2007.16104.pdf
#The Adam optimizer [50] with β1 = 0.9 and β2 = 0.999 and learning rate 5 × 10−4 was used.

#https://arxiv.org/pdf/2109.09559.pdf
#We used an Adam optimizer [85] with a cosine annealing learning rate scheduler and a three-time warm restart [86]. 
#The initial learning rate was set to 0.0007, and the weight decay was set to 0.015 empirically. 
'''
In the contrastive learning procedure, we used stratified
normalization [84] during training. In stratified normalization, we concatenated the same channel of different samples from one subject in the minibatch together and conducted z-score normalization. The stratified normalization
was applied to inputs of the base encoder, outputs of average pooling, and outputs of the temporal convolution in
the projector. F
'''

if str(device).startswith("cuda"):
    torch.set_float32_matmul_precision('high') #Increase performance
    
#Logger
project_name='Experiment_11'
import wandb
wandb.login()


#198d0b46f74d1861672c12c46489054b043161f2

True

##SimCLR Class

## PreTrext Task

The Pretext task is using contrastive learning with the Conformer as encoder, which creates vector representations of the eeg input. Those representations can be used in a downstream task, to analyse how well the representations capture the input information. A simple

In [4]:
print(device)
#transform_eeg = ContrastiveTransformations(device, probability_toZero=0.4)
def gaussian_noise(X, std):
      #https://github.com/braindecode/braindecode/blob/master/braindecode/augmentation/functional.py
      """Randomly add Gaussian noise to all channels.

      Suggested e.g. in [1]_, [2]_ and [3]_

      Parameters
      ----------
      X : torch.Tensor
          EEG input example or batch.
      y : torch.Tensor
          EEG labels for the example or batch.
      std : float
          Standard deviation to use for the additive noise.

      .. [1] Wang, F., Zhong, S. H., Peng, J., Jiang, J., & Liu, Y. (2018). Data
        augmentation for eeg-based emotion recognition with deep convolutional
        neural networks. In International Conference on Multimedia Modeling
        (pp. 82-93).
      .. [2] Cheng, J. Y., Goh, H., Dogrusoz, K., Tuzel, O., & Azemi, E. (2020).
        Subject-aware contrastive learning for biosignals. arXiv preprint
        arXiv:2007.04871.
      .. [3] Mohsenvand, M. N., Izadi, M. R., & Maes, P. (2020). Contrastive
        Representation Learning for Electroencephalogram Classification. In
        Machine Learning for Health (pp. 238-253). PMLR.
      """
      if not isinstance(X, torch.Tensor):
          X = torch.tensor(X)
          
      noise = torch.normal(0, std, size=(X.shape)) #.to(device)
      transformed_X = X + noise
      return transformed_X
        
def create_tuple(x):
    x1 = gaussian_noise(x, 0.4)
    x2 = gaussian_noise(x, 0.1)
    return (x1,x2)

cuda


In [ ]:
#SEED_IV

    
from torcheeg.datasets import SEEDIVFeatureDataset
SEEDiv_dataset_path = os.path.join(TMP_DATA, 'SEEDiv_FE')
SEEDivdataset = SEEDIVFeatureDataset(io_path=SEEDiv_dataset_path,
                      root_path=SEEDiv_PATH,
                      feature=['de_movingAve'],
                      offline_transform=transforms.Compose([
                                              transforms.BaselineRemoval(),
                                              transforms.To2d(),
                                              transforms.MeanStdNormalize(),
                                              ]),
                          online_transform=transforms.Compose([
                                                transforms.ToTensor(),
                                                transforms.Lambda(lambda x: create_tuple(x)),
                                                ]),
                                          label_transform=transforms.Compose([
                                              transforms.Select('emotion'),
                                              transforms.Lambda(lambda x: x + 1)]),
                                           num_worker=NUM_WORKERS
                                          )
print(SEEDivdataset[0])

In [ ]:
len(SEEDivdataset)

In [ ]:
from torch.utils.data import ConcatDataset
# Merge the datasets
Combined_dataset = ConcatDataset([SEEDivdataset, SEEDivdataset2])

In [ ]:
from torch.utils.data import ConcatDataset
test = 9
validation = 3
test_dataset = Datasets[test]
val_dataset = Datasets[validation]
train = [Datasets[i] for i in range(15) if i != test and i != validation]
train_dataset = ConcatDataset(train)

In [8]:
train_FE_SSL, test_FE_SSL = sklearn.model_selection.train_test_split(SEEDivdataset, test_size=0.05, train_size=None, random_state=None, shuffle=True, stratify=None)
print(f'Training Samples: {len(train_FE_SSL)}')
print(f'Training Samples: {len(test_FE_SSL)}')
#print(f'First Sample : {train_FE_SSL[0].shape}')

Training Samples: 35696
Training Samples: 1879


In [ ]:
train_FE_SSL[0][0][0].shape

In [ ]:
train_FE_SSL1 = train_FE_SSL[:-7]
print(f'new train len: {len(train_FE_SSL1)}')
print(f'old test len: {len(test_FE_SSL)}')
test_FE_SSL1 = ConcatDataset([train_FE_SSL[-7:], test_FE_SSL])
print(f'new test len: {len(test_FE_SSL1)}')

In [6]:
#from lightning.pytorch.callbacks.early_stopping import EarlyStopping

def train_simclr(model, num_samples, train_dataset, val_dataset, optimizer, temperature, lr, weight_decay, model_name = 'SimCLR', max_epochs=50, batch_size=100, embedding_dim=2048,feat_dim=128,logger=None, **kwargs):
        
    #default_root_dir=os.path.join(CHECKPOINT_PATH, 'version_0/'+'checkpoints/')
    print(f'model: {model_name}, \n default_root: {CHECKPOINT_PATH}')
    
    #Save Models in CHECKPOINT_PATH with model_name 
    save_model_callback = ModelCheckpoint(dirpath=CHECKPOINT_PATH,filename=model_name)
    #Accumlate the gradients: in this case from 0-4 epoch it will accumlate 8 batches, from 5-8 it will accumlate 4 and finally 1
    accumulator_callback = GradientAccumulationScheduler(scheduling={0: 8, 4: 4, 8: 1})
    #EarlyStopping: stop and skip the rest of the current epoch when val_loss cant be reduced anymore
    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min")
    #All Callsbacks:
    callbacks = [save_model_callback, LearningRateMonitor("epoch")]

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, model_name+'.ckpt')
    if os.path.isfile(pretrained_filename):
        print(f'Found pretrained model at {pretrained_filename}, loading...')
        # Automatically loads the model 
        ssl_model = SimCLR.load_from_checkpoint(pretrained_filename)
        
        print(f'Model loaded!')
        
    else:
        print(f'No pretrained model at {pretrained_filename}, start training ...')
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

        '''
        NOTE: The first element of each batch in :obj:`train_loader` and :obj:`val_loader` should be a two-tuple, representing two random transformations (views) of data. You can use :obj:`Contrastive` to achieve this functionality.
        '''
 
    
        
        ssl_model = SimCLR(model,
                           accelerator= 'auto', #"gpu" if str(device).startswith("cuda") else "cpu",
                           device='auto',
                            gpus = 1, 
                            num_samples=num_samples,
                            batch_size=batch_size,
                            #dataset: str,
                            num_nodes = 1,
                            hidden_mlp= embedding_dim, #h(x): hidden_representation for downstream 
                            feat_dim= feat_dim, #z(x) projection_representation 
                            warmup_epochs= 10,
                            max_epochs= max_epochs,
                            temperature = temperature,
                            rnoise = False,
                            optimizer= optimizer,
                            exclude_bn_bias = False,
                            start_lr= 0.0,
                            learning_rate= lr,
                            final_lr= 0.0,
                            weight_decay= weight_decay,
                            )
        
        
        ssl_model.fit(train_loader, val_loader, logger=logger, callbacks=callbacks)

    return ssl_model, pretrained_filename

In [ ]:
SSL_name='SSLExperiment_SeedIVOnly'
logger = WandbLogger(log_model="all", project=project_name, name=SSL_name)
#model_name_path='./saved_models/SSLExperiment_SEEDIV_norm-v1.ckpt'
model_name_path ='./saved_models/SSLExperiment_SeedIVOnly_augmented.ckpt'
model_name = 'SSLExperiment_SeedIVOnly_Augmented'
#pretrained_filename = os.path.join(CHECKPOINT_PATH, model_name+'.ckpt')
#Save Models in CHECKPOINT_PATH with model_name 
save_model_callback = ModelCheckpoint(dirpath=CHECKPOINT_PATH,filename=model_name)
#Accumlate the gradients: in this case from 0-4 epoch it will accumlate 8 batches, from 5-8 it will accumlate 4 and finally 1
accumulator_callback = GradientAccumulationScheduler(scheduling={0: 8, 4: 4, 8: 1})
#EarlyStopping: stop and skip the rest of the current epoch when val_loss cant be reduced anymore
early_stop_callback = EarlyStopping(monitor="val_loss", mode="min")
#All Callsbacks:
callbacks = [save_model_callback, accumulator_callback, LearningRateMonitor("epoch")]


train_loader = DataLoader(train_FE_SSL, batch_size=batch_sizeSSL, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(test_FE_SSL, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)

ssl_model = SimCLR.load_from_checkpoint(model_name_path)
ssl_model.fit(train_loader, val_loader, logger=logger, callbacks=callbacks)

In [ ]:
wandb.finish()

In [ ]:
SSL_name='SSLExperiment_SeedIVOnly_augmented_GaussianNoise'
wandb_logger = WandbLogger(log_model="all", project=project_name, name=SSL_name)
embedding_dim = 2048 #Embedding for Downstream
feat_dim = 128 #Projection_head
rnoise = False #With rolling Noise, if false, then GaussianNoise aus Augmentation

'''
TRAIN MODEL
'''
print(f'Start Training with Rnoise: {rnoise}, Epochs: {nr_epochs_ssl}, batch_size:{batch_sizeSSL}, embedding_dim: {embedding_dim}, lr: {lrSSL}, weight_decay:{weight_decaySSL}')

'''
model_ViT = ViT(chunk_size = 5,
                     grid_size = (9, 9),
                     t_patch_size = 1,
                     s_patch_size = (3, 3),
                     hid_channels = 32,
                     depth=3,
                     heads= 6,
                     head_channels = 32,
                     mlp_channels = 32,
                     num_classes= embedding_dim,
                     embed_dropout= 0.,
                     dropout = 0.2,
                     pool_func = 'cls')
'''
model_ViT   =    ArjunViT(chunk_size=5,
                 t_patch_size=1,
                 num_electrodes=channels,
                 num_classes=embedding_dim)

#A common observation in contrastive learning is that the larger the batch size, the better the models perform. A larger batch size allows us to compare each image to more negative examples, leading to overall smoother loss gradients.
simclr_model, pretrained_filename = train_simclr(model =model_ViT,
                                                num_samples = len(train_FE_SSL),
                                                train_dataset = train_FE_SSL,
                                                val_dataset = test_FE_SSL, 
                                                model_name = SSL_name,
                                                max_epochs=nr_epochs_ssl,
                                                batch_size=batch_sizeSSL,
                                                embedding_dim=embedding_dim,
                                                feat_dim=feat_dim, #For the projection_head 
                                                lr=lrSSL,
                                                temperature=temperatureSSL,
                                                weight_decay=weight_decaySSL,
                                                rnoise = rnoise,
                                                devices = 'auto',
                                                logger=wandb_logger,
                                                optimizer= "lars",
                                                )

wandb.finish()

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [4]:
#print(pretrained_filename)
#print(pretrained_filename)
#pretrained_filename = './saved_models/SSLExperiment_BIGData-v3.ckpt'
pretrained_filename = './saved_models/SSLExperiment_SeedIVOnly_augmented_newRollingNoise-v2.ckpt'
#pretrained_filename = './saved_models/SSLExperiment_SeedIVOnly_augmented_GaussianNoise-v1.ckpt'

In [5]:
#AFTER TRAINING
##Load Model_1 with weights and parameters:
print(pretrained_filename)

if os.path.isfile(pretrained_filename):
        print(f'Found pretrained model at {pretrained_filename}, loading...')  
        pretrained_ssl_model_1 = SimCLR.load_from_checkpoint(pretrained_filename)
else:
    print(f'No Model found') 

./saved_models/SSLExperiment_SeedIVOnly_augmented_newRollingNoise-v2.ckpt
Found pretrained model at ./saved_models/SSLExperiment_SeedIVOnly_augmented_newRollingNoise-v2.ckpt, loading...


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [ ]:
#!rm -rf './Tmp_Datasets/SEED_PATH_Subj8'
#!rm -rf './Tmp_Datasets/SEED_PATH_Subj7'

## Downstream task

The Pretext task was trained using contrastive learning with the Conformer as encoder, which creates vector representations of the eeg input. Those representations can be used in a downstream task, to analyse how well the representations capture the input information. A simple Logistic Regression can be used to classify the encoder vectors. For that we use the pretrained weights and create a new non-augmented Dataset. The non-augmented EEG samples are passed to the pretrained encoder to optain the input representations for the logistic Regression. The performance of the LR demonstrates how good the representations are. 

In [ ]:
#Leave one out

from torcheeg.datasets import SEEDFeatureDataset
SEED_PATH_Subj = 'SEED_PATH_Subj'
Datasets = []
for i in range(1,16):
    name = SEED_PATH_Subj + str(i)
    file_path = SEED_PATH + str(i)
    print(name)
    
    SEED_dataset_path = os.path.join(TMP_DATA, name)
    Datasets.append(SEEDFeatureDataset(
                      io_path=SEED_dataset_path, #location of loaded and transformed dataset
                      root_path=file_path, #location of original dataset 
                      feature=['de_movingAve'],
                      offline_transform=transforms.Compose([
                                              transforms.BaselineRemoval(),
                                              transforms.ToTensor(),
                                              transforms.To2d(),
                                              transforms.MeanStdNormalize(),
                                              ]),
                                          label_transform=transforms.Compose([
                                              transforms.Select('emotion'),
                                              transforms.Lambda(lambda x: x + 1)]),
                                           num_worker=NUM_WORKERS
                                          ))
print(Datasets)

In [ ]:
from torch.utils.data import ConcatDataset
subject_test_int = 15
subject_train = 'train_AllSubj'
subject_test = 'train_Subj'+str(subject_test_int)

test_dataset = Datasets[subject_test_int-1]
#val_dataset = Datasets[validation-1]
train = [Datasets[i] for i in range(15) if i != subject_test_int-1]
train_dataset = ConcatDataset(train)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
disk_info = !df -h
disk_info = '\n'.join(disk_info)
print(disk_info)

In [ ]:
pretrained_ssl_model_1.projection.model

In [13]:
@torch.no_grad()
def prepare_data_features(model, dataset):
    # Prepare model
    network = deepcopy(model)
    # Removing projection head g(.)
    
    for x in range(4):
        network.projection.model[x]=nn.Identity()
    
    network.eval()
    network.to(device)
    
    # Encode all EEG samples by passing them through the pretrained SSL model
    data_loader = DataLoader(dataset, batch_size=64, shuffle=False)
    feats, labels = [], []
    for batch_eegs, batch_labels in tqdm(data_loader):
        batch_eegs = batch_eegs.squeeze()
        #print(len(batch_imgs))
        batch_eegs = batch_eegs.to(device)
        batch_feats = network(batch_eegs)
        feats.append(batch_feats.detach().cpu())
        labels.append(batch_labels)

    feats = torch.cat(feats, dim=0)
    labels = torch.cat(labels, dim=0)
    print(feats.shape)
    feats = feats.unsqueeze(1) 
    print(feats.shape)

    return TensorDataset(feats, labels)

In [ ]:
#Generate 1-D Feature Vector for Downstreamtask based on pretrained SSL-model
train_feats_simclr = prepare_data_features(pretrained_ssl_model_1, train_dataset)
val_feats_simclr = prepare_data_features(pretrained_ssl_model_1, test_dataset)
#test_feats_simclr = prepare_data_features(pretrained_ssl_model_1, test_dataset)
#print(f'generated features {train_feats_simclr}')
torch.save(train_feats_simclr, os.path.join(DATA_PATH, subject_train))
torch.save(val_feats_simclr, os.path.join(DATA_PATH, subject_test))
#torch.save(test_feats_simclr, os.path.join(DATA_PATH, 'test_Leave1Out_Experiment2'))
print(f'Saved to: {DATA_PATH}')

In [ ]:
print(os.path.join(DATA_PATH, 'test_feats_simclr'))

In [ ]:
#Load generated features of normal non_augmented Data
train_feats_simclr= torch.load(os.path.join(DATA_PATH, './'+subject_train))
val_feats_simclr= torch.load(os.path.join(DATA_PATH, './'+subject_test))


In [ ]:
val_feats_simclr= torch.load(os.path.join(DATA_PATH, './test_Leave1Out_Experiment2'))

In [5]:
def train_supervised(model, batch_size, train_data, test_data, model_name, num_classes, logger=None,  max_epochs=100, **kwargs):
    
    
    #Save Models in CHECKPOINT_PATH with model_name 
    save_model_callback = ModelCheckpoint(dirpath=CHECKPOINT_PATH,filename=model_name)
    #Accumlate the gradients: in this case from 0-4 epoch it will accumlate 8 batches, from 5-8 it will accumlate 4 and finally 1
    accumulator_callback = GradientAccumulationScheduler(scheduling={0: 8, 4: 4, 8: 1})
    #EarlyStopping: stop and skip the rest of the current epoch when val_loss cant be reduced anymore
    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min")
    
    print(f'model: {model_name}, \n default_root: {CHECKPOINT_PATH}')
    trainer = pl.Trainer(
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices='auto',
                         max_epochs=max_epochs,
                         callbacks=[save_model_callback,
                                    LearningRateMonitor("epoch"),
                                    accumulator_callback,
                                    early_stop_callback,
                                   ],
                         #enable_progress_bar=False,
                         check_val_every_n_epoch=5,
                         logger=logger,
                        )
    

    # Data loaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True,
                                   drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False,
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)

    print(f'Batch_size: {batch_size}')
    print(f'train_loader: {len(train_loader)}')

    model = Trainer_class2(model, lr, weight_decay, max_epochs, num_classes)
    trainer.fit(model, train_loader, test_loader)
    #model = Trainer_class2.load_from_checkpoint(pretrained_filename, model=model)
    print(f'Done with model: {model_name}')
    

In [ ]:
#Train with X number of samples per label instead of 100%: 
def get_smaller_dataset(original_dataset, num_eegs_per_label):
    new_dataset = TensorDataset(
        *[t.unflatten(0, (1, -1))[:,:num_eegs_per_label].flatten(0, 1) for t in original_dataset.tensors]
    )
    return new_dataset

In [ ]:
'''
Test the extraction of labels from the 'get_smaller_dataset' function
Check, how many labels are returned
'''
print(f'Total Size of train_dataset: {len(train_feats_simclr)}')
print(f'Total Size of test_dataset: {len(val_feats_simclr)}')

print(f'Extract only portion of data samples per label')
test = get_smaller_dataset(train_feats_simclr, 1425)
print(len(test))
print(f'One Sample is:')
print(test[0][0].shape)

In [ ]:
'''
Check label distribution of loaded data
'''
#Print Feature Infos of Data Samples 
train_t = train_feats_simclr.tensors[0]
#test = test.unsqueeze(1)
print(f'Training data: {train_t.shape}')
feature_dim=train_feats_simclr.tensors[0].shape[2]
list_labels_train = train_feats_simclr.tensors[1]
list_labels_test = val_feats_simclr.tensors[1]
print(f'Number of labels train: {len(list_labels_train)}, Number of labels test: {len(list_labels_test)}')
# Get the unique values and their counts from the concatenated tensor
unique_values_train, counts_train = torch.unique(list_labels_train, return_counts=True)
print(f'unique Labels train: {unique_values_train}, Counts: {counts_train}')
unique_values_test, counts_test = torch.unique(list_labels_test, return_counts=True)
print(f'unique Labels test: {unique_values_test}, Counts: {counts_test}')

In [ ]:
wandb.finish()

In [ ]:
from torch.utils.data import ConcatDataset



print(f'Start Training with Epochs: {nr_epochs}, nr_classes: {nr_classes}, batch_size:{batch_size}, lr: {lr}, weight_decay:{weight_decay}')

#get 10%, 50% and 100% of the labels per sample:
#_, counts_train = torch.unique(list_labels_train, return_counts=True)
#percentages = [int(counts_train[0]*0.1), int(counts_train[0]*0.5), int(counts_train[0])]
#print(f'train on using: {percentages} amount of labels')


for subject_test_int in [1,2,3,15,13]:

    name_ds = "GausianNoise_DS_SEED_p"+str(subject_test_int)
    print(f'current model: {name_ds}')
    
    pretrained_filename = os.path.join(CHECKPOINT_PATH, name_ds+'-v1.ckpt')
    print(pretrained_filename)
    if os.path.isfile(pretrained_filename):
        print(f'Model {name_ds} already exist')
        continue
    else:
        #wandb_logger = WandbLogger(log_model="all", project=project_name, name=name_ds)
    
        subject_train = 'GausianNoise_train_AllSubj-'+str(subject_test_int)
        subject_test = 'GausianNoise_train_Subj'+str(subject_test_int)
        
        test_dataset = Datasets[subject_test_int-1]
        train = [Datasets[i] for i in range(15) if i != subject_test_int-1]
        train_dataset = ConcatDataset(train)
    
        #Generate 1-D Feature Vector for Downstreamtask based on pretrained SSL-model
        train_feats_simclr = prepare_data_features(pretrained_ssl_model_1, train_dataset)
        val_feats_simclr = prepare_data_features(pretrained_ssl_model_1, test_dataset)
        
        #test_feats_simclr = prepare_data_features(pretrained_ssl_model_1, test_dataset)
        #print(f'generated features {train_feats_simclr}')
        torch.save(train_feats_simclr, os.path.join(DATA_PATH, subject_train))
        torch.save(val_feats_simclr, os.path.join(DATA_PATH, subject_test))
    
        #Get feature Dimension 
        feature_dim=train_feats_simclr.tensors[0].shape[2]

        model = nn.Sequential(
            nn.Linear(in_features=feature_dim, out_features=nr_classes)  
            )
        
        #Get only X-% of samples per label:
        #sub_train_set = get_smaller_dataset(train_feats_simclr, num_eegs_per_label)


        
        print(f'Start Training on all, except subj: {subject_test_int}')
        print(f'Model_name: {name_ds}')
        
        train_supervised(model = model, 
                        batch_size=batch_size,
                        max_epochs=nr_epochs,
                        train_data=train_feats_simclr,
                        test_data=val_feats_simclr,
                        model_name=f'{name_ds}',
                        num_classes=nr_classes,
                        lr=lr,
                        weight_decay=weight_decay,
                        logger = None,
                        )
        #wandb.finish()

In [8]:
from sklearn.metrics import confusion_matrix
from torchmetrics import F1Score
import seaborn as sn
import pandas as pd

def get_results_per_model(model, test_loader):


    y_pred = []
    y_true = []

    
    # iterate over test data
    for inputs, labels in test_loader:
            #inputs = inputs.squeeze()
            output = model.model(inputs) # Feedforward pass
            output = output.squeeze()
            output = torch.argmax(output, dim=1, keepdim=True).numpy()

            y_pred.extend(output) # Save Prediction
            
            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth

    return y_pred,y_true

def get_confusion_matrix(y_pred,y_true):
    
     # constant for classes
    classes = ('Negative', 'Neutral', 'Positive')
    
    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                         columns = [i for i in classes])
    plt.figure(figsize = (12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig(model_name+'_output.png')

    #F1 Score:
    f1 = F1Score(task="multiclass", num_classes=3)
    print('F1Score:')
    y_pred = torch.tensor(y_pred).squeeze()
    F1 = f1(y_pred,torch.tensor(y_true))
    print(F1)
    return F1, df_cm, cf_matrix

In [ ]:
import re
#SSL DS:
#[1,2,3,15,13]
#Rolling Noise
model_1 ='./saved_models/DS_SEED_p1-v1.ckpt'
model_2 ='./saved_models/DS_SEED_p2-v1.ckpt'
model_3 ='./saved_models/DS_SEED_p3-v1.ckpt'
model_15 ='./saved_models/DS_SEED_p15-v1.ckpt'
model_13 ='./saved_models/DS_SEED_p13-v1.ckpt'

model_list = [model_1, model_2, model_3, model_15, model_13]


trainer = pl.Trainer(
                     accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                     devices='auto',
                     max_epochs=nr_epochs,
                    )



ypred_all = []


mode = 'RollingNoise'


for model_name in model_list:
    print(f'model: {model_name[-15:]}')
    try: 
        model = nn.Sequential(
                            nn.Linear(in_features=2048, out_features=nr_classes)  
                            )
        model = Trainer_class2.load_from_checkpoint(model_name, model=model)
        print(f'loaded model successfully')
    
    except Exception as e:
        # If an exception occurs, print an error message
        print(f"Error loading model for {model_name}: {e}")

    #Get Test data    
    try:
        # Extract the number from the file name (get the integer between 'p' and '-v1').
        extracted_number = int(re.search(r'p(\d+)-v1', model_name).group(1))
        print(f'extraced number: {extracted_number}')  
        # Try to load the file
        if mode == 'RollingNoise':
            test_feats_simclr = torch.load(os.path.join(DATA_PATH, f'./train_Subj{extracted_number}')) #SSL_features_subj is a dataset
        else:
            test_feats_simclr = torch.load(os.path.join(DATA_PATH, f'./GausianNoise_train_Subj{extracted_number}'))
    
        print(f'Mode: {mode}')
        
        # If successful:
        print(f"File for Subject {extracted_number} loaded successfully.")
    
    except Exception as e:
        # If an exception occurs, print an error message
        print(f"Error loading file for Subject {model_name}: {e}")
    
    
    #Get feature Dimension 
    feature_dim=test_feats_simclr.tensors[0].shape[2]
    
    test_loader = DataLoader(test_feats_simclr, batch_size=batch_size, shuffle=False,
                              drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)


    # Test best model on test set
    test_result = trainer.test(model, test_loader, verbose=False)
    print(f'test_result: {test_result}')
    y_pred,y_true = get_results_per_model(model, test_loader)
    
    F1, df_cm, cf_matrix = get_confusion_matrix(y_pred,y_true)
    print(f'F1: {F1}')
    print(f'df_cm: {df_cm}')
    ypred_all.append(df_cm)

In [ ]:
import re
#SSL DS:
#[1,2,3,15,13]
#Rolling Noise
model_1 ='./saved_models/DS_SEED_p1-v1.ckpt'
model_2 ='./saved_models/DS_SEED_p2-v1.ckpt'
model_3 ='./saved_models/DS_SEED_p3-v1.ckpt'
model_15 ='./saved_models/DS_SEED_p15-v1.ckpt'
model_13 ='./saved_models/DS_SEED_p13-v1.ckpt'

#AverageNoise

model_1 = './saved_models/GausianNoise_DS_SEED_p1-v1.ckpt'
model_2 = './saved_models/GausianNoise_DS_SEED_p2-v1.ckpt'
model_3 = './saved_models/GausianNoise_DS_SEED_p3-v1.ckpt'
model_15 = './saved_models/GausianNoise_DS_SEED_p15-v1.ckpt'
model_13 = './saved_models/GausianNoise_DS_SEED_p13-v1.ckpt'


model_list = [model_1, model_2, model_3, model_15, model_13]


trainer = pl.Trainer(
                     accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                     devices='auto',
                     max_epochs=nr_epochs,
                    )



ypred_all = []


mode = 'GaussianNoise'


for model_name in model_list:
    print(f'model: {model_name[-15:]}')
    try: 
        model = nn.Sequential(
                            nn.Linear(in_features=2048, out_features=nr_classes)  
                            )
        model = Trainer_class2.load_from_checkpoint(model_name, model=model)
        print(f'loaded model successfully')
    
    except Exception as e:
        # If an exception occurs, print an error message
        print(f"Error loading model for {model_name}: {e}")

    #Get Test data    
    try:
        # Extract the number from the file name (get the integer between 'p' and '-v1').
        extracted_number = int(re.search(r'p(\d+)-v1', model_name).group(1))
        print(f'extraced number: {extracted_number}')  
        # Try to load the file
        if mode == 'RollingNoise':
            test_feats_simclr = torch.load(os.path.join(DATA_PATH, f'./train_Subj{extracted_number}')) #SSL_features_subj is a dataset
        else:
            test_feats_simclr = torch.load(os.path.join(DATA_PATH, f'./GausianNoise_train_Subj{extracted_number}'))
    
        print(f'Mode: {mode}')
        
        # If successful:
        print(f"File for Subject {extracted_number} loaded successfully.")
    
    except Exception as e:
        # If an exception occurs, print an error message
        print(f"Error loading file for Subject {model_name}: {e}")
    
    
    #Get feature Dimension 
    feature_dim=test_feats_simclr.tensors[0].shape[2]
    
    test_loader = DataLoader(test_feats_simclr, batch_size=batch_size, shuffle=False,
                              drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)


    # Test best model on test set
    test_result = trainer.test(model, test_loader, verbose=False)
    print(f'test_result: {test_result}')
    y_pred,y_true = get_results_per_model(model, test_loader)
    
    F1, df_cm, cf_matrix = get_confusion_matrix(y_pred,y_true)
    print(f'F1: {F1}')
    print(f'df_cm: {df_cm}')
    ypred_all.append(df_cm)

In [ ]:
# Take the average of the DataFrames
average_df = pd.concat(ypred_all).groupby(level=0).mean()
print(f'Average over all k-folds: {average_df}')
plt.figure(figsize = (12,7))
sn.heatmap(average_df, annot=True)
plt.savefig('Average_kfold_ds_GaussianNoise.png')

## Visualise Embeddings

Visualise the embeddings of the self-supervised learning model

In [ ]:
#Load embeddings
subject = 15
SSL_features_subj = torch.load(os.path.join(DATA_PATH, f'./train_Subj{subject}'))
print(SSL_features_subj[0][0].shape)

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


#Load one subject
subject = 15
subjectlist = [1,2,3,15,13]

for subject in subjectlist:
    print(f'model_name number: {subject}')
    for mode in ['GaussianNoise', 'RollingNoise']:
    
        if mode == 'RollingNoise':
            SSL_features_subj = torch.load(os.path.join(DATA_PATH, f'./train_Subj{subject}')) #SSL_features_subj is a dataset
        else:
            SSL_features_subj = torch.load(os.path.join(DATA_PATH, f'./GausianNoise_train_Subj{subject}'))
    
        print(f'Mode: {mode}')
        # Extract training vectors and labels
        train_vectors = []
        labels = []
        
        for instance in SSL_features_subj:
            train, label = instance
            train_vectors.append(train.numpy().flatten())
            labels.append(label)
        
        train_vectors = np.array(train_vectors)
        labels = np.array(labels)
        
        # Apply t-SNE to the training vectors
        tsne = TSNE(n_components=2, random_state=42)
        train_vectors_tsne = tsne.fit_transform(train_vectors)
        
        # Create a scatter plot with colored points based on labels
        plt.scatter(train_vectors_tsne[:, 0], train_vectors_tsne[:, 1], c=labels, cmap='viridis')
        plt.title(f't-SNE Visualization of Subj-{subject} for {mode}')
        plt.xlabel('t-SNE Component 1')
        plt.ylabel('t-SNE Component 2')
        plt.colorbar(label='Labels')
        plt.show()
        plt.savefig(f't-SNE_subj{subject}_{mode}.png')


In [25]:
pretrained_ssl_model_1.projection.model

Sequential(
  (0): Linear(in_features=2048, out_features=2048, bias=True)
  (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=2048, out_features=128, bias=False)
)

In [ ]:
from torch.utils.data import ConcatDataset



print(f'Start Training with Epochs: {nr_epochs}, nr_classes: {nr_classes}, batch_size:{batch_size}, lr: {lr}, weight_decay:{weight_decay}')

#get 10%, 50% and 100% of the labels per sample:
#_, counts_train = torch.unique(list_labels_train, return_counts=True)
#percentages = [int(counts_train[0]*0.1), int(counts_train[0]*0.5), int(counts_train[0])]
#print(f'train on using: {percentages} amount of labels')


for subject_test_int in [1,2,3,15,13]:

    name_ds = "FineTune_Rolling_DS_SEED_p"+str(subject_test_int)
    print(f'current model: {name_ds}')
    
    pretrained_filename = os.path.join(CHECKPOINT_PATH, name_ds+'-v1.ckpt')
    print(pretrained_filename)
    if os.path.isfile(pretrained_filename):
        print(f'Model {name_ds} already exist')
        continue
    else:
        #wandb_logger = WandbLogger(log_model="all", project=project_name, name=name_ds)

        test_dataset = Datasets[subject_test_int-1]
        train = [Datasets[i] for i in range(15) if i != subject_test_int-1]
        train_dataset = ConcatDataset(train)

        network = deepcopy(pretrained_ssl_model_1)
        #Changing the last layer to [x, nr_classes]
        network.projection.model[0]=nn.Linear(in_features=2048, out_features=nr_classes) 
        for x in range(1,4):
            network.projection.model[x] == nn.Identity()
            
        print(f'Start Training on all, except subj: {subject_test_int}')
        print(f'Model_name: {name_ds}')

        train_supervised(model = network, 
                        batch_size=batch_size,
                        max_epochs=nr_epochs,
                        train_data=train_dataset,
                        test_data=test_dataset,
                        model_name=f'{name_ds}',
                        num_classes=nr_classes,
                        lr=lr,
                        weight_decay=weight_decay,
                        logger = None,
                        )
        #wandb.finish()

    

In [ ]:
#Perform statisticall test
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import mcnemar
import re

#Load models that used rollingnoise
model_1 ='./saved_models/DS_SEED_p1-v1.ckpt'
model_2 ='./saved_models/DS_SEED_p2-v1.ckpt'
model_3 ='./saved_models/DS_SEED_p3-v1.ckpt'
model_15 ='./saved_models/DS_SEED_p15-v1.ckpt'
model_13 ='./saved_models/DS_SEED_p13-v1.ckpt'

model_list_rollingnoise = [model_1, model_2, model_3, model_15, model_13]

#Load models that used GaussianNoise
model_1_gn = './saved_models/GausianNoise_DS_SEED_p1-v1.ckpt'
model_2_gn = './saved_models/GausianNoise_DS_SEED_p2-v1.ckpt'
model_3_gn = './saved_models/GausianNoise_DS_SEED_p3-v1.ckpt'
model_15_gn = './saved_models/GausianNoise_DS_SEED_p15-v1.ckpt'
model_13_gn = './saved_models/GausianNoise_DS_SEED_p13-v1.ckpt'


model_list_averagenoise = [model_1_gn, model_2_gn, model_3_gn, model_15_gn, model_13_gn]

def get_results_per_model2(model, test_loader):


    y_pred = []
    y_true = []

    
    # iterate over test data
    for inputs, labels in test_loader:
            inputs = inputs.squeeze()
            inputs = inputs.to(device)
            output = model.model(inputs) # Feedforward pass
            output = output.squeeze()
            output = torch.argmax(output.cpu(), dim=1, keepdim=True).numpy()

            y_pred.extend(output) # Save Prediction
            
            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth

    return y_pred,y_true


for rolling_noise_model, guassian_noise_model in zip(model_list_rollingnoise, model_list_averagenoise):
    print('test models:')
    print(rolling_noise_model)
    print(guassian_noise_model)


    #load model 1:
    print(f'model: {rolling_noise_model[-15:]}')
    extracted_number = int(re.search(r'p(\d+)-v1', rolling_noise_model).group(1))
    print(f'extraced number: {extracted_number}')
    try: 
        model = nn.Sequential(
                            nn.Linear(in_features=2048, out_features=nr_classes)  
                            )
        model = Trainer_class2.load_from_checkpoint(rolling_noise_model, model=model)
        print(f'loaded model1 successfully')

    except Exception as e:
            # If an exception occurs, print an error message
            print(f"Error loading model for {rolling_noise_model}: {e}")    

    test_feats_rolling_noise = torch.load(os.path.join(DATA_PATH, f'./train_Subj{extracted_number}')) #SSL_features_subj is a dataset


    #load model 2:
    try: 
        model2 = nn.Sequential(
                            nn.Linear(in_features=2048, out_features=nr_classes)  
                            )
        model2 = Trainer_class2.load_from_checkpoint(guassian_noise_model, model=model2)
        print(f'loaded model2 successfully')

    except Exception as e:
            # If an exception occurs, print an error message
            print(f"Error loading model for {guassian_noise_model}: {e}")
    
    test_feats_guassian_noise = torch.load(os.path.join(DATA_PATH, f'./GausianNoise_train_Subj{extracted_number}'))

    #Get feature Dimension 
    #feature_dim=test_feats_guassian_noise.tensors[0].shape[2]
        
    test_loader_1 = DataLoader(test_feats_rolling_noise, batch_size=batch_size, shuffle=False,
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    test_loader_2 = DataLoader(test_feats_guassian_noise, batch_size=batch_size, shuffle=False,
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)

    # Test best model on test set
    y_pred1,y_true_1 = get_results_per_model2(model, test_loader_1)
    y_pred2,y_true_2 = get_results_per_model2(model2, test_loader_2)

    assert np.array_equal(y_true_1, y_true_2), "y_true_2 is not the same as y_true_1"   

    y_true_1 = np.array(y_true_1)
    y_pred1 = np.array(np.squeeze(y_pred1))
    y_pred2 = np.array(np.squeeze(y_pred2))

    tb = mcnemar_table(y_target=y_true_1, 
                   y_model1=y_pred1, 
                   y_model2=y_pred2)

    
    print(tb)
    

    chi2, p = mcnemar(ary=tb, corrected=True)
    print('chi-squared:', chi2)
    print('p-value:', p)    
    if p < 0.05:
        print('Models are significant different')
    else:
        print('Models are NOT significant different!')

In [ ]:
#Perform statisticall test
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import mcnemar
import re

#Load models that used rollingnoise
model_1 ='./saved_models/DS_SEED_p1-v1.ckpt'
model_2 ='./saved_models/DS_SEED_p2-v1.ckpt'
model_3 ='./saved_models/DS_SEED_p3-v1.ckpt'
model_15 ='./saved_models/DS_SEED_p15-v1.ckpt'
model_13 ='./saved_models/DS_SEED_p13-v1.ckpt'

model_list_rollingnoise = [model_1, model_2, model_3, model_15, model_13]

#Load models that used GaussianNoise
model_1_bl = './saved_models/Basline_SEED_p1-v1.ckpt'
model_2_bl = './saved_models/Basline_SEED_p2-v1.ckpt'
model_3_bl = './saved_models/Basline_SEED_p3-v1.ckpt'
model_15_bl = './saved_models/Basline_SEED_p15-v1.ckpt'
model_13_bl = './saved_models/Basline_SEED_p13-v1.ckpt'


model_list_baseline = [model_1_bl, model_2_bl, model_3_bl, model_15_bl, model_13_bl]

def get_results_per_model2(model, test_loader):


    y_pred = []
    y_true = []

    
    # iterate over test data
    for inputs, labels in test_loader:
            inputs = inputs.squeeze()
            inputs = inputs.to(device)
            output = model.model(inputs) # Feedforward pass
            output = output.squeeze()
            output = torch.argmax(output.cpu(), dim=1, keepdim=True).numpy()

            y_pred.extend(output) # Save Prediction
            
            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth

    return y_pred,y_true


for rolling_noise_model, baseline_model in zip(model_list_rollingnoise, model_list_baseline):
    print('test models:')
    print(rolling_noise_model)
    print(baseline_model)


    #load model 1:
    print(f'model: {rolling_noise_model[-15:]}')
    extracted_number = int(re.search(r'p(\d+)-v1', rolling_noise_model).group(1))
    print(f'extraced number: {extracted_number}')
    try: 
        model = nn.Sequential(
                            nn.Linear(in_features=2048, out_features=nr_classes)  
                            )
        model = Trainer_class2.load_from_checkpoint(rolling_noise_model, model=model)
        print(f'loaded model1 successfully')

    except Exception as e:
            # If an exception occurs, print an error message
            print(f"Error loading model for {rolling_noise_model}: {e}")    

    test_feats_rolling_noise = torch.load(os.path.join(DATA_PATH, f'./train_Subj{extracted_number}')) #SSL_features_subj is a dataset


    #load model 2:
    try: 
        model2 = nn.Sequential(
                            nn.Linear(in_features=2048, out_features=nr_classes)  
                            )
        model2 = Trainer_class2.load_from_checkpoint(baseline_model, model=model2)
        print(f'loaded model2 successfully')

    except Exception as e:
            # If an exception occurs, print an error message
            print(f"Error loading model for {baseline_model}: {e}")
    
    test_feats_guassian_noise = torch.load(os.path.join(DATA_PATH, f'./GausianNoise_train_Subj{extracted_number}'))

    #Get feature Dimension 
    #feature_dim=test_feats_guassian_noise.tensors[0].shape[2]
        
    test_loader_1 = DataLoader(test_feats_rolling_noise, batch_size=batch_size, shuffle=False,
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    test_loader_2 = DataLoader(test_feats_guassian_noise, batch_size=batch_size, shuffle=False,
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)

    # Test best model on test set
    y_pred1,y_true_1 = get_results_per_model2(model, test_loader_1)
    y_pred2,y_true_2 = get_results_per_model2(model2, test_loader_2)

    assert np.array_equal(y_true_1, y_true_2), "y_true_2 is not the same as y_true_1"   

    y_true_1 = np.array(y_true_1)
    y_pred1 = np.array(np.squeeze(y_pred1))
    y_pred2 = np.array(np.squeeze(y_pred2))

    tb = mcnemar_table(y_target=y_true_1, 
                   y_model1=y_pred1, 
                   y_model2=y_pred2)

    
    print(tb)
    

    chi2, p = mcnemar(ary=tb, corrected=True)
    print('chi-squared:', chi2)
    print('p-value:', p)    
    if p < 0.05:
        print('Models are significant different')
    else:
        print('Models are NOT significant different!')